In [97]:
from gurobipy import *

In [98]:
##### Model #####

model = Model('model')


##### Parameters #####

''' Adjust size of the parking lot '''
B = 120.0       # Width of parking lot
L = 120.0       # Length of parking lot (Length of exterior rows)
w = 7.0         # Width of the road at top and bottom
l = L - 2*w     # Length of interior rows
'''--------------------------------'''

''' Adjust angles of a parking spot row '''   
num_of_angles = 5
angle_set = [90, 75, 60, 45, 30]
'''-------------------------------------''' 

''' Change parameters here if angles changed '''
A2 = [2.5, 2.7, 2.8, 3.5, 5.6] # Bay width for car, parallel to aisle
A3 = [1.0, 1.08, 1.12, 1.4, 2.24]  # Bay width for motorcylcle, parallel to aisle -> A2*0.4

C1 = [5.0, 5.5, 5.6, 5.3, 4.5] # Bay depth to wall for car
C3 = [5.0, 5.15, 5.05, 4.40, 3.40] # Bay depth to interlock for car
C4 = [6.0, 6.5, 6.6, 6.3, 5.5] # Bay depth to wall for electric car -> C1+1
C5 = [6.0, 6.15, 6.05, 5.40, 4.40] # Bay depth to interlock for electric car -> C3+1
C6 = [2.0, 2.2, 2.24, 2.12, 1.8] # Bay depth to wall for motorcylcle -> C1*0.4
C7 = [2.0, 2.06, 2.02, 1.76, 1.36] # Bay depth to interlock for motorcylcle -> C3*0.4

D1  = [7.0, 6.0, 4.5, 3.75, 3.5] # Aisle width between bay lines for car
D2  = [2.8, 2.4, 1.8, 1.5, 1.4] # Aisle width between bay lines for motorcycle -> D1*0.4

F1 = [18.0, 17.75, 15.65, 13.45, 11.4] # Module, wall to interlock (F1 = C1+D1+C3)
F3 = [18.0, 17.4, 14.8, 12.55, 10.3] # Module, interlock to interlock (F3 = C3+D1+C3)

#### Variables

In [99]:
# angle set = [90, 75, 60, 45, 30]
X = [] # number of full interiors rows for car
Xe = [] # number of full exteriors rows for car
E = [] # number of exteriors rows for car
XBev = [] # number of full interiors rows for electric car
XeBev = [] 
EBev = []
XMot = [] # number of full interiors rows for motorcycle
XeMot = []
EMot = []

n = []
ne = []
nEE = []
nBev = [] # number of full interiors bays for electric car
neBev = []
nEEBev = []
nMot = []  # number of full interiors bays for motorcycle
neMot = []
nEEMot = []

for i in range(num_of_angles):
    X.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='X'+str(angle_set[i])))
    Xe.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='Xe'+str(angle_set[i])))
    E.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='E'+str(angle_set[i])))
    XBev.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='XBev'+str(angle_set[i])))
    XeBev.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='XeBev'+str(angle_set[i])))
    EBev.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='EBev'+str(angle_set[i])))
    XMot.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='XMot'+str(angle_set[i])))
    XeMot.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='XeMot'+str(angle_set[i])))
    EMot.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='EMot'+str(angle_set[i])))   
    
    n.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='n'+str(angle_set[i])))
    ne.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='ne'+str(angle_set[i])))
    nEE.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='nEE'+str(angle_set[i])))
    nBev.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='nBev'+str(angle_set[i])))
    neBev.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='neBev'+str(angle_set[i])))
    nEEBev.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='nEEBev'+str(angle_set[i])))
    nMot.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='nMot'+str(angle_set[i])))
    neMot.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='neMot'+str(angle_set[i])))
    nEEMot.append(model.addVar(lb=0, vtype=GRB.INTEGER, name='nEEMot'+str(angle_set[i])))    

# z = 1 if a row is for car
z = model.addVar(lb=0, vtype = GRB.BINARY, name = "z")
# ze = model.addVar(lb=0, vtype = GRB.BINARY, name = "ze")
# zEE = model.addVar(lb=0, vtype = GRB.BINARY, name = "zEE")
# zBev = 1 if a row is for electric car
zBev = model.addVar(lb=0, vtype = GRB.BINARY, name = "zBev")
# zeBev = model.addVar(lb=0, vtype = GRB.BINARY, name = "zeBev")
# zEEBev = model.addVar(lb=0, vtype = GRB.BINARY, name = "zEEBev")
# zMot = 1 if a row is for motorcycle
zMot = model.addVar(lb=0, vtype = GRB.BINARY, name = "zMot")
# zeMot = model.addVar(lb=0, vtype = GRB.BINARY, name = "zeMot")
# zEEMot = model.addVar(lb=0, vtype = GRB.BINARY, name = "zEEMot")

In [100]:
# Maximize the total number of parking spots
model.setObjective(
    quicksum(n[i] + ne[i] + nEE[i] + 1.4*(nBev[i] + neBev[i] + nEEBev[i]) + 0.18*(nMot[i]+ neMot[i]+ nEEMot[i]) for i in range(num_of_angles)),
    GRB.MAXIMIZE)

#### Constraints

In [101]:
# Total width of full interior rows, full exterior rows, and exterior rows should be less than the width of  the parking lot
model.addConstr(quicksum((2*C3[i]+D1[i])*X[i] + (2*C7[i]+D2[i])*XMot[i] + (2*C5[i]+D1[i])*XBev[i] \
    + (C1[i]+D1[i]+C3[i])*Xe[i] + (C6[i]+D2[i]+C7[i])*XeMot[i] + (C4[i]+D1[i]+C5[i])*XeBev[i] \
    + (C1[i]+D1[i])*E[i] + (C6[i]+D1[i])*EMot[i] + (C4[i]+D1[i])*EBev[i] \
    for i in range(num_of_angles)) <= B)

# The number of spots in each rows should be less than its upper bound
# Upper bound = (max length of row / width of a spot) * number of rows
model.addConstrs(l//A2[i]*2*X[i] >= n[i] for i in range(num_of_angles))
model.addConstrs(l//A2[i]*Xe[i] + L//A2[i]*Xe[i] >= ne[i] for i in range(num_of_angles))
model.addConstrs(L//A2[i]*E[i] >= nEE[i] for i in range(num_of_angles))

model.addConstrs(l//A2[i]*2*XBev[i] >= nBev[i] for i in range(num_of_angles))
model.addConstrs(l//A2[i]*XeBev[i] + L//A2[i]*XeBev[i] >= neBev[i] for i in range(num_of_angles))
model.addConstrs(L//A2[i]*EBev[i] >= nEEBev[i] for i in range(num_of_angles))

model.addConstrs(l//A3[i]*2*XMot[i] >= nMot[i] for i in range(num_of_angles))
model.addConstrs(l//A3[i]*XeMot[i] + L//A3[i]*XeMot[i] >= neMot[i] for i in range(num_of_angles))
model.addConstrs(L//A3[i]*EMot[i] >= nEEMot[i] for i in range(num_of_angles))

# There should be exactly two exterior rows
model.addConstr(quicksum(Xe[i] + E[i] + XeBev[i] + EBev[i] + XeMot[i] + EMot[i]for i in range(num_of_angles)) == 2)


<gurobi.Constr *Awaiting Model Update*>

In [102]:
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 47 rows, 93 columns and 165 nonzeros
Model fingerprint: 0xac853812
Variable types: 0 continuous, 93 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+02]
Found heuristic solution: objective 68.0000000
Presolve removed 45 rows and 67 columns
Presolve time: 0.00s
Presolved: 2 rows, 26 columns, 40 nonzeros
Found heuristic solution: objective 756.2400000
Variable types: 0 continuous, 26 integer (0 binary)

Root relaxation: objective 7.637590e+02, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   G

#### Results

In [103]:
results = open('LP_results_new.csv', 'w')

results.write(f'B, {B}')
results.write(f'\nL, {L}')
results.write(f'\nw, {w}')

results.write(f'\nobj. value, {model.objVal}')

results.write(f'\nvar\\angle')
for i in range(num_of_angles):
    results.write(f',{angle_set[i]}')
    
results.write(f'\nX')
for i in range(num_of_angles):
    results.write(f',{X[i].x}')
    
results.write(f'\nXe')
for i in range(num_of_angles):
    results.write(f',{Xe[i].x}')
    
results.write(f'\nE')
for i in range(num_of_angles):
    results.write(f',{E[i].x}')

results.write(f'\nXBev')
for i in range(num_of_angles):
    results.write(f',{XBev[i].x}')
    
results.write(f'\nXeBev')
for i in range(num_of_angles):
    results.write(f',{XeBev[i].x}')
    
results.write(f'\nEBev')
for i in range(num_of_angles):
    results.write(f',{EBev[i].x}')   

results.write(f'\nXMot')
for i in range(num_of_angles):
    results.write(f',{XMot[i].x}')
    
results.write(f'\nXeMot')
for i in range(num_of_angles):
    results.write(f',{XeMot[i].x}')
    
results.write(f'\nEMot')
for i in range(num_of_angles):
    results.write(f',{EMot[i].x}')

results.write(f'\nn')
for i in range(num_of_angles):
    results.write(f',{n[i].x}')
    
results.write(f'\nne')
for i in range(num_of_angles):
    results.write(f',{ne[i].x}')
    
results.write(f'\nnEE')
for i in range(num_of_angles):
    results.write(f',{nEE[i].x}')

results.write(f'\nnBev')
for i in range(num_of_angles):
    results.write(f',{nBev[i].x}')
    
results.write(f'\nneBev')
for i in range(num_of_angles):
    results.write(f',{neBev[i].x}')
    
results.write(f'\nnEEBev')
for i in range(num_of_angles):
    results.write(f',{nEEBev[i].x}')

results.write(f'\nnMot')
for i in range(num_of_angles):
    results.write(f',{nMot[i].x}')
    
results.write(f'\nneMot')
for i in range(num_of_angles):
    results.write(f',{neMot[i].x}')
    
results.write(f'\nnEEMot')
for i in range(num_of_angles):
    results.write(f',{nEEMot[i].x}')

results.close()